In [44]:
%matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
# import gmplot
# from geopy.geocoders import Nominatim
# import time
# import gmaps
# import gmaps.datasets

Aijobsindustry_dataset = pd.read_csv("AIJobsIndustry_Clean_v2.csv")
Aijobsindustry_dataset

Dice_us_jobs_dataset = pd.read_csv("Dice_US_jobs_Clean_v2.csv", encoding="cp437")
Dice_us_jobs_dataset

monster_dataset = pd.read_csv("Monster_Clean.csv")
monster_dataset

##file need cleaning
# alldata_dataset = pd.read_csv("alldata.csv")
# alldata_dataset

##file needs to match format for concat
# Dice_use_Tech_filtered = pd.read_csv("Dice_US_Tech_FilteredJobData.csv")
# Dice_use_Tech_filtered

datasets_to_merge = [Aijobsindustry_dataset,Dice_us_jobs_dataset,monster_dataset]

##please confirm the concat makes sense
merged_dataset = pd.concat(datasets_to_merge)
merged_dataset


Using matplotlib backend: Qt5Agg


,Position,Company,Description,Loc_state,Loc_city,Skills,Type,Source
0,Data Science Development Program- Junior Data ...,\r\n<<<<<<< HEAD\r\nMassMutual,NaN,MA,Amherst,NaN,NaN,NaN
1,Entry-Level Cognitive / AI / Machine Learning ...,\r\nIBM,NaN,TX,Austin,NaN,NaN,NaN
2,PHP/MySQL Programmer,NaN,NaN,TX,Austin,NaN,NaN,NaN
3,Python Programmer Job,\r\nSAIC,NaN,CA,Monterey,NaN,NaN,NaN
4,BI Analyst,NaN,NaN,CA,San Francisco,NaN,NaN,NaN
5,L2- Jr. SQL Developer,"\r\nGartner, Inc.",NaN,NY,New York,NaN,NaN,NaN
6,BI Analyst,NaN,NaN,CA,San Francisco,NaN,NaN,NaN
7,Data Engineer,\r\nApple,NaN,CA,Santa Clara Valley,NaN,NaN,NaN
8,Supply Chain Data Scientist/Analyst,\r\nApple,NaN,CA,Santa Clara Valley,NaN,NaN,NaN
9,Data Insights Analyst (Advanced SQL Needed),\r\nC.H. Robinson,NaN,MN,Eden Prairie,NaN,NaN,NaN


In [ ]:
def parser(column):
    word_count = {}
    for entry in column:
        entry = re.sub('[^A-Za-z0-9 ]+', '', str(entry))
#        print(entry)
        try:
            entry = entry.replace(',','')
            word_split = entry.split()
            for word in word_split:
                if word not in word_count:
                    word_count[word] = 1
                else:
                    word_count[word] += 1
        except AttributeError:
            print("Skipping an entry due to bad data")
            continue
    print(word_count)

In [ ]:
desc = data1['description']
parser(desc)

In [ ]:
group_by_state = monster_dataset.groupby("Loc_state")
count_for_state = group_by_state.count()
count_for_state = count_for_state["Position"]
count_for_state